In [1]:
%cd ../

/Users/Matteo/PycharmProjects/TrafficEmu


In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator
import experimental_design.config as config

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [4]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [5]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route duration
        result.append(s['timeLoss']/(s['duration'] * s['speed']))
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model Inputs / parameter space

In [6]:
parameter_space = config.get_parameter_space()

### Get init, test, and train points

Train points are used to optimize a GP without experimental design - to verify experimental design works.

In [7]:
design = RandomDesign(parameter_space)
init_X = design.get_samples(20)
init_Y = user_function_time_loss(init_X)

design = RandomDesign(parameter_space)
test_X = design.get_samples(50)
test_Y = user_function_time_loss(test_X)

design = RandomDesign(parameter_space)
train_X = design.get_samples(270)
train_Y = user_function_time_loss(train_X)

[[17.         16.54672711 26.65024964 49.39868362  1.          2.04534632]
 [14.          8.10154659 20.31951646 43.07129858  3.          3.12082837]
 [11.         19.81438307  9.45902677 45.34608534  2.          1.548178  ]
 [10.         12.30842156 25.85690788 63.67883874  1.          1.69950228]
 [ 7.         17.48670123 15.93596633 38.73726654  3.          3.19783345]
 [19.         24.58968644 26.95614182 32.35683821  2.          2.50847927]
 [16.         19.10368982 21.58606951 52.46944389  1.          2.23003101]
 [14.         16.24791738 11.04390841 45.72242476  3.          2.93171126]
 [ 8.         21.12240215 24.90754974 64.13936886  1.          4.02442867]
 [ 7.         24.2507225  12.74328701 53.75709243  3.          4.879222  ]
 [ 9.         19.60221509 16.81528477 65.03782558  2.          3.92135173]
 [ 8.         13.96060615  6.98971681 61.53613441  1.          4.27260238]
 [ 7.         18.69104509 25.30115725 68.38728907  1.          3.5784463 ]
 [15.          8.73304137


Evaluating input: 36 of 50

 Retrying in 1 seconds

Evaluating input: 37 of 50

 Retrying in 1 seconds

Evaluating input: 38 of 50

 Retrying in 1 seconds

Evaluating input: 39 of 50

 Retrying in 1 seconds

Evaluating input: 40 of 50

 Retrying in 1 seconds

Evaluating input: 41 of 50

 Retrying in 1 seconds

Evaluating input: 42 of 50

 Retrying in 1 seconds

Evaluating input: 43 of 50

 Retrying in 1 seconds

Evaluating input: 44 of 50

 Retrying in 1 seconds

Evaluating input: 45 of 50

 Retrying in 1 seconds

Evaluating input: 46 of 50

 Retrying in 1 seconds

Evaluating input: 47 of 50

 Retrying in 1 seconds

Evaluating input: 48 of 50

 Retrying in 1 seconds

Evaluating input: 49 of 50

 Retrying in 1 seconds

Evaluating input: 50 of 50

 Retrying in 1 seconds
[[12.         19.03810381 21.70634426 33.17166425  3.          2.07380182]
 [18.         18.306412   15.8701729  61.6371389   2.          3.61948402]
 [ 9.         10.80773887 18.16207099 53.45559076  3.          3.07138


Evaluating input: 131 of 270

 Retrying in 1 seconds

Evaluating input: 132 of 270

 Retrying in 1 seconds

Evaluating input: 133 of 270

 Retrying in 1 seconds

Evaluating input: 134 of 270

 Retrying in 1 seconds

Evaluating input: 135 of 270

 Retrying in 1 seconds

Evaluating input: 136 of 270

 Retrying in 1 seconds

Evaluating input: 137 of 270

 Retrying in 1 seconds

Evaluating input: 138 of 270

 Retrying in 1 seconds

Evaluating input: 139 of 270

 Retrying in 1 seconds

Evaluating input: 140 of 270

 Retrying in 1 seconds

Evaluating input: 141 of 270

 Retrying in 1 seconds

Evaluating input: 142 of 270

 Retrying in 1 seconds

Evaluating input: 143 of 270

 Retrying in 1 seconds

Evaluating input: 144 of 270

 Retrying in 1 seconds

Evaluating input: 145 of 270

 Retrying in 1 seconds

Evaluating input: 146 of 270

 Retrying in 1 seconds

Evaluating input: 147 of 270

 Retrying in 1 seconds

Evaluating input: 148 of 270

 Retrying in 1 seconds

Evaluating input: 149 of 27

### Experimental Design - Model Variance

#### Model / Emulator (GP)

Our surrogate model is our emulator. In this case, a gaussian process.

In [8]:
emulator_ed = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ed = GPyModelWrapper(emulator_ed, n_restarts=5)
emukit_model_ed.optimize()
emulator_ed

Optimization restart 1/5, f = -67.43039929354521
Optimization restart 2/5, f = -67.43039929552418
Optimization restart 3/5, f = -38.54329430666736
Optimization restart 4/5, f = -38.54329430667104
Optimization restart 5/5, f = -67.43039929583743


GP_regression.,value,constraints,priors
rbf.variance,0.002072345887009046,+ve,
rbf.lengthscale,83.7681143877531,+ve,
Gaussian_noise.variance,3.1138197083820515e-05,+ve,


#### Acquisition Function

In [9]:
model_variance = ModelVariance(model=emukit_model_ed)

#### Optimizer

In [10]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [11]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ed,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [12]:
ed_loop.run_loop(user_function_time_loss, 250)

Optimization restart 1/5, f = -67.43039929588755
Optimization restart 2/5, f = -38.54329430667199
Optimization restart 3/5, f = -38.54329430670175
Optimization restart 4/5, f = -67.43039929589975
Optimization restart 5/5, f = -67.43039929590742
[[ 3.         12.82763838  6.11290235 30.97717003  3.          3.80292813]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -71.22121042811158
Optimization restart 2/5, f = -40.58170147119566
Optimization restart 3/5, f = -40.581701467256885
Optimization restart 4/5, f = -40.58170146725469
Optimization restart 5/5, f = -40.581728760892595
[[20.         24.26626349  6.27598189 67.36861932  2.          2.26794857]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -75.1673871205529
Optimization restart 2/5, f = -75.16738712057006
Optimization restart 3/5, f = -42.876616513

Optimization restart 5/5, f = -74.3871603368514
[[ 4.          9.2361118   5.50674863 65.15163655  1.          4.22739248]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -128.71020863999274
Optimization restart 2/5, f = -75.99214990935306
Optimization restart 3/5, f = -75.99214863780715
Optimization restart 4/5, f = -128.71020864071647
Optimization restart 5/5, f = -128.71020862657167
[[ 5.          8.52946041  6.47746218 69.54051412  3.          1.73236055]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -132.47345689537153
Optimization restart 2/5, f = -112.68798021651392
Optimization restart 3/5, f = -78.27382913648472
Optimization restart 4/5, f = -132.4738164285173
Optimization restart 5/5, f = -132.47381643959426
[[ 3.          8.23929817 25.91542834 32.88519784  2.          2.26095521]]

User functi

 Retrying in 1 seconds
Optimization restart 1/5, f = -177.69871158104988
Optimization restart 2/5, f = -177.69871158102154
Optimization restart 3/5, f = -103.49806598264487
Optimization restart 4/5, f = -103.49805105134526
Optimization restart 5/5, f = -177.69871158105025
[[20.         24.83246216 11.7025949  37.98500272  1.          2.2105675 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -180.21696337239516
Optimization restart 2/5, f = -180.21696337262634
Optimization restart 3/5, f = -105.49803700963348
Optimization restart 4/5, f = -180.21696337263046
Optimization restart 5/5, f = -105.49804040549499
[[ 4.         24.64810243 25.24780244 62.02718173  3.          1.75798666]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -177.64036174730273
Optimization restart 2/5, f = -105.99204893947238
Optimizat

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = -177.64036174711697
[[19.          8.4877469  26.55726955 40.38610888  3.          3.81991154]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -181.44885801567267
Optimization restart 2/5, f = -181.44885801547275
Optimization restart 3/5, f = -108.04809812797937
Optimization restart 4/5, f = -181.44885801567867
Optimization restart 5/5, f = -181.4488580156791
[[17.         12.27758528 14.70953553 69.90496422  3.          4.59756338]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -184.50228852802528
Optimization restart 2/5, f = -110.17968784972928
Optimization restart 3/5, f = -184.5022885300943
Optimization restart 4/5, f = -184.50228852865058
Optimization restart 5/5, f = -184.5022885285112
[[ 4.          8.338702   13.95734041 33.11187782  1.          1.63185738]]

User fun

...

Optimization restart 4/5, f = -794.1805094353024
Optimization restart 5/5, f = -794.1805094107549
[[ 3.          8.53887813  5.21301452 42.23750365  3.          3.84284204]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -798.0957556017709
Optimization restart 2/5, f = -798.095792222384
Optimization restart 3/5, f = -798.0957922756623
Optimization restart 4/5, f = -448.50648706022423
Optimization restart 5/5, f = -448.50683852386055
[[15.          8.5937108  11.22746564 30.04444633  2.          1.81347191]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -800.6093880843799
Optimization restart 2/5, f = -450.61365867944266


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 3/5, f = -800.6093880864173
Optimization restart 4/5, f = -800.6093880861503
Optimization restart 5/5, f = -800.6093880814298
[[20.          8.62102332  5.0155728  69.46639805  3.          2.7422401 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -804.5471675880619
Optimization restart 2/5, f = -452.56203823123474
Optimization restart 3/5, f = -452.56195047238845
Optimization restart 4/5, f = -452.5621204575308
Optimization restart 5/5, f = -804.5471675935239
[[20.         24.97373684 19.39646037 41.82605268  3.          1.60730239]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -808.4681279220258
Optimization restart 2/5, f = -808.4682165873071
Optimization restart 3/5, f = -454.03948536895865
Optimization restart 4/5, f = -454.03948537067276
Optimization restart 5/5, f = -808.46821

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 2/5, f = -784.3878574210314
Optimization restart 3/5, f = -814.232988753228
Optimization restart 4/5, f = -814.2329887907646
Optimization restart 5/5, f = -458.0234967606702
[[20.         22.78110011 26.69658966 69.62542404  3.          4.91917306]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -817.938637890179
Optimization restart 2/5, f = -460.01294606564585
Optimization restart 3/5, f = -817.9386439048176
Optimization restart 4/5, f = -460.01294606564784
Optimization restart 5/5, f = -817.9386453987834
[[ 9.          8.12914721  5.27771526 31.93717349  3.          1.80113147]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -819.7196860537495
Optimization restart 2/5, f = -819.7196860546666
Optimization restart 3/5, f = -819.7196859749063
Optimization restart 4/5, f = -461.578659112

### Experimental Design - Integrated Variance Reduction

#### Model / Emulator (GP)

In [13]:
emulator_ird = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ird = GPyModelWrapper(emulator_ird, n_restarts=5)
emukit_model_ird.optimize()

Optimization restart 1/5, f = -67.43039929354521
Optimization restart 2/5, f = -38.54329437168947
Optimization restart 3/5, f = -38.54329430667272
Optimization restart 4/5, f = -67.43039929590384
Optimization restart 5/5, f = -38.54329430667198


#### Acquisition Function

In [14]:
integrated_variance_reduction = IntegratedVarianceReduction(
    model=emukit_model_ird, 
    space=parameter_space, 
    num_monte_carlo_points=100
)

#### Optimizer

In [15]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [16]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ird,
    space = parameter_space,
    acquisition = integrated_variance_reduction,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [17]:
ed_loop.run_loop(user_function_time_loss, 250)

Optimization restart 1/5, f = -67.43039929590384
Optimization restart 2/5, f = -67.43039929583665
Optimization restart 3/5, f = -38.54329430667198
Optimization restart 4/5, f = -38.54329430667198
Optimization restart 5/5, f = -38.54329430667198
[[ 5.         11.58872212  5.99043443 30.00883739  2.          3.27685996]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -71.20946055495767
Optimization restart 2/5, f = -40.38804659759489
Optimization restart 3/5, f = -40.38804659759487
Optimization restart 4/5, f = -71.20946055407197
Optimization restart 5/5, f = -40.388046597594865
[[20.         15.78620219  9.78726906 69.12174661  1.          3.50868374]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -75.25710399242175
Optimization restart 2/5, f = -42.71016914744808
Optimization restart 3/5, f = -42.710169147

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = -131.84006786508985
[[20.         21.13453383 14.40460809 46.28110445  1.          3.26180362]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -136.08621041835897
Optimization restart 2/5, f = -75.61322445292028
Optimization restart 3/5, f = -136.0862104214174
Optimization restart 4/5, f = -75.6130759770798
Optimization restart 5/5, f = -75.61307593031523
[[ 3.         10.2593166   5.1291574  48.02172484  3.          4.87547831]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -131.2160799476294
Optimization restart 2/5, f = -131.21607994701537


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 3/5, f = -127.31944928132862
Optimization restart 4/5, f = -76.6445524502221
Optimization restart 5/5, f = -131.21607994762076
[[ 3.          9.48719648 25.33907796 55.59619342  3.          2.59676086]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -133.67191479358263
Optimization restart 2/5, f = -78.84155829074797
Optimization restart 3/5, f = -131.94439387777211
Optimization restart 4/5, f = -78.8415583058123
Optimization restart 5/5, f = -78.8415582907518
[[20.         10.75985935 16.47334607 56.51903646  3.          2.99373789]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -137.72152997105215
Optimization restart 2/5, f = -137.72152997114435
Optimization restart 3/5, f = -81.03380303591264
Optimization restart 4/5, f = -137.72152997116353
Optimization restart 5/5, f = -137.72152

...

Optimization restart 4/5, f = -757.2801342611822
Optimization restart 5/5, f = -768.7860588213034
[[ 3.         17.74544385 26.99631822 63.13486     1.          4.8468495 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -772.6162094574837
Optimization restart 2/5, f = -430.72926883561513
Optimization restart 3/5, f = -430.7292728016615
Optimization restart 4/5, f = -430.7292688694475
Optimization restart 5/5, f = -772.6167269542259
[[20.         24.27545351 26.58660538 43.36757953  1.          1.55870956]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -776.4103739735102
Optimization restart 2/5, f = -776.4104521563569
Optimization restart 3/5, f = -776.4104521525351
Optimization restart 4/5, f = -776.4104521558932
Optimization restart 5/5, f = -776.4104521563527
[[10.         15.48714662 21.55892371 44.10

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 2/5, f = -823.9876756361762
Optimization restart 3/5, f = -463.065802049779
Optimization restart 4/5, f = -827.9216393663904
Optimization restart 5/5, f = -827.9216392779564
[[ 3.         11.10164981 12.00819124 35.74523947  3.          4.9855956 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -830.8138122272103
Optimization restart 2/5, f = -465.00980089812464
Optimization restart 3/5, f = -709.8278784183763
Optimization restart 4/5, f = -830.8138122185965
Optimization restart 5/5, f = -830.8138121919378
[[14.         20.44435825 26.29125675 30.44564645  3.          2.4842664 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -834.4330694866426
Optimization restart 2/5, f = -834.4330694870378
Optimization restart 3/5, f = -834.4330694811518
Optimization restart 4/5, f = -834.433069487

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 2/5, f = -835.8999557290584
Optimization restart 3/5, f = -846.0126621974962
Optimization restart 4/5, f = -846.0126621950936
Optimization restart 5/5, f = -471.36509782657583
[[14.          8.01889895 15.31481639 32.00037208  1.          4.96250813]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -849.8347294995328
Optimization restart 2/5, f = -849.8347405154428
Optimization restart 3/5, f = -849.8347405154785
Optimization restart 4/5, f = -849.8347405165757
Optimization restart 5/5, f = -473.31879721082936
[[20.         20.13861249 19.12064368 44.81978578  3.          1.50927049]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -853.3088732610246
Optimization restart 2/5, f = -853.3088732699964
Optimization restart 3/5, f = -474.8048560659156
Optimization restart 4/5, f = -523.8814133

### Random GP

Supervised learning

In [18]:
emulator_rnd = GPy.models.GPRegression(train_X, train_Y)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=50)
emukit_model_rnd.optimize()
emulator_rnd

Optimization restart 1/50, f = -960.918285427231
Optimization restart 2/50, f = -511.62741978109585
Optimization restart 3/50, f = -960.9182854551038
Optimization restart 4/50, f = -960.9182854540743
Optimization restart 5/50, f = -511.6274077783679
Optimization restart 6/50, f = -511.6274047256805
Optimization restart 7/50, f = -960.918285420086
Optimization restart 8/50, f = -960.9182854551186
Optimization restart 9/50, f = -960.9182854521168
Optimization restart 10/50, f = -960.9182854478145
Optimization restart 11/50, f = -960.9182854549638
Optimization restart 12/50, f = -960.9182854550697
Optimization restart 13/50, f = -511.6274047394052
Optimization restart 14/50, f = -960.9182854549883
Optimization restart 15/50, f = -960.9182854548042
Optimization restart 16/50, f = -960.9182854549969
Optimization restart 17/50, f = -960.9182854538171
Optimization restart 18/50, f = -960.9182853683776


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 19/50, f = -950.1615392299846
Optimization restart 20/50, f = -511.6274047799458
Optimization restart 21/50, f = -960.9182854551166
Optimization restart 22/50, f = -960.9182854290888
Optimization restart 23/50, f = -960.9182854540213
Optimization restart 24/50, f = -960.9182853502728
Optimization restart 25/50, f = -960.9182854548933
Optimization restart 26/50, f = -960.9182854504681
Optimization restart 27/50, f = -511.6274167340782
Optimization restart 28/50, f = -960.9182854323303
Optimization restart 29/50, f = -511.62780951942483
Optimization restart 30/50, f = -511.6274047256444
Optimization restart 31/50, f = -511.627405169756
Optimization restart 32/50, f = -960.9182854487765
Optimization restart 33/50, f = -960.9182854551025
Optimization restart 34/50, f = -511.62748091487185
Optimization restart 35/50, f = -960.918285446656
Optimization restart 36/50, f = -511.62740472563985
Optimization restart 37/50, f = -511.6276665837537
Optimization restart 38/50, f 

GP_regression.,value,constraints,priors
rbf.variance,0.003634446409291793,+ve,
rbf.lengthscale,18.35178016315764,+ve,
Gaussian_noise.variance,2.036382614924786e-05,+ve,


### Evaluate

Evaluate RMSE of each model on our test set of 50 randomly selected points.

In [19]:
pred_Y, pred_Y_variance = emukit_model_ed.predict(test_X)
mse_ed = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Model Variance RMSE: ', mse_ed)

Experimental Design w/ Model Variance RMSE:  0.008533233040113455


In [20]:
pred_Y, pred_Y_variance = emukit_model_ird.predict(test_X)
mse_ird = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Integrated Variance Reduction RMSE: ', mse_ird)

Experimental Design w/ Integrated Variance Reduction RMSE:  0.0060967586281770545


In [21]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse_rnd = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Random GP RMSE: ', mse_rnd)

Random GP RMSE:  0.006403011859194119


In [22]:
p = 1 - (mse_ed/mse_rnd)
f'Experimental Design w/ Model Variance is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Model Variance is -33.269% better than training on random points.'

In [23]:
p = 1 - (mse_ird/mse_rnd)
f'Experimental Design w/ Integrated Variance Reduction is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Integrated Variance Reduction is 4.783% better than training on random points.'

### Save Models

In [25]:
with open(f'experimental_design_new_outputs/models/model_variance_20_init_points_250_loops_timeloss.pkl', "wb") as f:
     pickle.dump(emukit_model_ed, f)

with open(f'experimental_design_new_outputs/models/integrated_variance_reduction_20_init_points_250_loops_timeloss.pkl', "wb") as f:
     pickle.dump(emukit_model_ird, f)
        
with open(f'experimental_design_new_outputs/models/270_random_points_supervised_GP_timeloss.pkl', "wb") as f:
     pickle.dump(emukit_model_rnd, f)